In [1]:
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
data = pd.read_feather('target_stocks.feather')
print(data)

         index                stock_list
0   2022-10-21                    [1717]
1   2022-10-28              [6873, 1904]
2   2022-11-03                    [2520]
3   2022-11-04                    [6412]
4   2022-11-09              [6188, 1409]
..         ...                       ...
154 2024-09-23                    [4919]
155 2024-09-24                    [2634]
156 2024-09-25                    [2606]
157 2024-09-27                    [3260]
158 2024-09-30  [6491, 8150, 9907, 6592]

[159 rows x 2 columns]


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import logging
import yaml
from daily_bt import (
    load_config, 
    read_stock_data, 
    construct_data, 
    execute_short_strategy,
    calculate_daily_performance,
    backtest_single_stock,
    backtest_multiple_stocks, 
    get_stocks_for_date, 
    main
)


In [4]:
config = load_config('config.yaml')

In [5]:
df= read_stock_data('2606', datetime.strptime('2023-11-16', '%Y-%m-%d'), '/Users/mouyasushi/k_data/永豐')
print(df.head())
print(df.tail())
print(len(df))   # 每一隻股票的 length 會不一樣 （skip emtpy trade ) 

                      Open   High    Low  Close  Volume     Amount
ts                                                                
2023-11-16 09:01:00  45.00  45.05  44.80  44.80      80  3597900.0
2023-11-16 09:02:00  44.80  44.80  44.50  44.55      88  3925600.0
2023-11-16 09:03:00  44.55  44.75  44.55  44.75      77  3438750.0
2023-11-16 09:04:00  44.80  44.95  44.80  44.95      41  1837950.0
2023-11-16 09:05:00  45.00  45.00  44.90  44.90      14   629200.0
                      Open   High    Low  Close  Volume     Amount
ts                                                                
2023-11-16 13:22:00  45.05  45.15  45.05  45.15      79  3562000.0
2023-11-16 13:23:00  45.10  45.10  45.05  45.10      32  1443150.0
2023-11-16 13:24:00  45.10  45.10  45.05  45.05       9   405500.0
2023-11-16 13:25:00  45.05  45.10  45.00  45.00      39  1756200.0
2023-11-16 13:30:00  45.05  45.05  45.05  45.05     190  8559500.0
241


In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
np_data= construct_data(df, ma_period=14) # this is a dictioinary 
open_np = np_data['Open']
print(open_np)
print(len(open_np))


[45.   44.8  44.55 44.8  45.   44.9  44.95 45.1  45.   44.85 44.8  44.85
 44.8  44.85 44.9  44.95 44.95 44.95 44.95 44.85 44.85 44.85 44.85 44.85
 44.9  44.9  44.85 44.8  44.8  44.75 44.8  44.75 44.8  44.75 44.75 44.75
 44.7  44.6  44.6  44.45 44.55 44.5  44.55 44.55 44.55 44.6  44.6  44.65
 44.7  44.65 44.65 44.7   0.   44.7  44.65 44.5  44.5   0.   44.5  44.5
 44.45 44.5  44.5  44.45 44.5  44.45 44.45 44.45 44.45 44.5  44.45 44.45
 44.5  44.5  44.45 44.45 44.5  44.7  44.75 44.75 44.75 44.65 44.65 44.65
 44.65 44.7  44.75  0.   44.75 44.8  44.75 44.8  44.75 44.8  44.8  44.8
 44.75  0.   44.75 44.75 44.8  44.95 45.   44.95 44.95 45.   45.05 45.
 45.   45.   45.   45.    0.   44.95 45.   45.   44.95 44.95 45.   45.
 44.95 45.   45.   45.   45.   45.   45.   45.   45.05 45.   44.95  0.
 44.95  0.   44.95 44.95 44.95 45.    0.   44.95 45.   45.05 45.05 45.05
  0.   45.1  45.05 45.05 45.05 45.05 45.05  0.   45.05 45.05 45.05 45.
  0.    0.   45.    0.   45.   45.05 45.   45.05 45.05 45.05 

/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:102: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range(start=trading_start, end=trading_end, freq='1T')
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Volume'].replace(0, np.nan, inplace=True)
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:118: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignm

In [7]:
""" 
execute_short_strategy(open_np, close_np, vwap_np, open_time_np, initial_cap, fee_rate, 
                         stop_loss_pct, threshold, entry_time_min, previous_day_last_close) --> equity, trade times
"""

equity = execute_short_strategy(
    np_data['Open'],
    np_data['Close'],
    np_data['VWAP'],
    np_data['open_time'],
    initial_cap=100000,
    fee_rate=0.001,
    stop_loss_pct=0.04,
    threshold=1.01,
    entry_time_min=546,  # 9:06 AM
    previous_day_last_close=30)

equity

(array([100000, 100000, 100000, 100000, 100000,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688,  99688,
         99688,  99688,  99688,  99688,  99688,  99688,  99688, 

In [8]:
calculate_daily_performance(equity[0], trade_count=equity[1]) 

{'Total Trades': 1,
 'Total Return': -0.0031200000000000117,
 'Max Drawdown': 0.0031200000000000117,
 'Win Rate': 0.0,
 'Daily Volatility': 0.00018987584125087026}

In [9]:
data.head()

,index,stock_list
0,2022-10-21,[1717]
1,2022-10-28,"[6873, 1904]"
2,2022-11-03,[2520]
3,2022-11-04,[6412]
4,2022-11-09,"[6188, 1409]"


In [10]:
data

,index,stock_list
0,2022-10-21,[1717]
1,2022-10-28,"[6873, 1904]"
2,2022-11-03,[2520]
3,2022-11-04,[6412]
4,2022-11-09,"[6188, 1409]"
...,...,...
154,2024-09-23,[4919]
155,2024-09-24,[2634]
156,2024-09-25,[2606]
157,2024-09-27,[3260]


In [11]:
print(type(data['index']))
print(type(data['stock_list']))
print(type(data['stock_list'][0]))
print(type(data['stock_list'][0][0]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'str'>


In [12]:
import logging

logging.basicConfig(level=logging.DEBUG)


from datetime import datetime

date_to_check = datetime.strptime('2022-10-21', '%Y-%m-%d')
stocks = get_stocks_for_date('target_stocks.feather', date_to_check)
print(f"Stocks for {date_to_check.date()}: {stocks}")


2024-10-23 10:08:18,199 - INFO - [daily_bt.py:252] - get_stocks_for_date - Reading stock list from Feather file: target_stocks.feather
2024-10-23 10:08:18,204 - INFO - [daily_bt.py:299] - get_stocks_for_date - Found 1 stocks for date 2022-10-21


Stocks for 2022-10-21: ['1717']


In [13]:
backtest_single_stock('1717', date_to_check, config)

2024-10-23 10:08:18,208 - INFO - [daily_bt.py:317] - backtest_single_stock - Running backtest for stock: 1717 on date: 2022-10-21
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:102: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range(start=trading_start, end=trading_end, freq='1T')
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Volume'].replace(0, np.nan, inplace=True)
/Users/mouyasushi/Deskto

{'Total Trades': 0,
 'Total Return': 0.0,
 'Max Drawdown': 0.0,
 'Win Rate': 0.0,
 'Daily Volatility': 0.0,
 'Stock': '1717',
 'Date': datetime.date(2022, 10, 21)}

In [14]:
stocks

['1717']

In [16]:
backtest_multiple_stocks(stocks, date_to_check, config)

2024-10-23 10:08:24,411 - INFO - [daily_bt.py:317] - backtest_single_stock - Running backtest for stock: 1717 on date: 2022-10-21
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:102: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range(start=trading_start, end=trading_end, freq='1T')
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Volume'].replace(0, np.nan, inplace=True)
/Users/mouyasushi/Deskto

[{'Total Trades': 0,
  'Total Return': 0.0,
  'Max Drawdown': 0.0,
  'Win Rate': 0.0,
  'Daily Volatility': 0.0,
  'Stock': '1717',
  'Date': datetime.date(2022, 10, 21)}]

In [17]:
main('config.yaml', '2022-10-21', 'target_stocks.feather')

2024-10-23 10:08:49,927 - INFO - [daily_bt.py:252] - get_stocks_for_date - Reading stock list from Feather file: target_stocks.feather
2024-10-23 10:08:49,935 - INFO - [daily_bt.py:299] - get_stocks_for_date - Found 1 stocks for date 2022-10-21
2024-10-23 10:08:50,390 - INFO - [daily_bt.py:317] - backtest_single_stock - Running backtest for stock: 1717 on date: 2022-10-21
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:102: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range(start=trading_start, end=trading_end, freq='1T')
/Users/mouyasushi/Desktop/sr8/setup/daily_bt.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value